<a href="https://colab.research.google.com/github/kr7/LogReg/blob/main/20250113LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Importieren der nötigen Bibliotheken**



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from google.colab import widgets
from sklearn.linear_model import LogisticRegression

**Vorbeireitung der Daten**

Wir laden die Daten:


In [ ]:
data_with_labels = np.loadtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', delimiter=',')

Wir laden auch die Namen der Spalten (damit wir wissen was für Information in den einzelnen Spaten gespeichert ist):

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names

In [ ]:
column_names = []
with open("spambase.names") as f:
  for i in range(33):
    f.readline()

  for i in range(57):
    line = f.readline()
    column_names.append(line.split(":")[0])

Wir teilen die Daten in Trainings- und Testdaten auf:

In [ ]:
data = data_with_labels[:,:-1]
labels = data_with_labels[:,-1]

test_indices = np.array(range(int(len(data)/5)))*5+4
train_indices = [i for i in range(len(data)) if i not in test_indices]

data_train0  = data[train_indices]
data_test0   = data[test_indices]
labels_train = labels[train_indices]
labels_test  = labels[test_indices]

Normalisierung der Daten:

In [ ]:
avg = np.mean(data_train0, axis=0)
std = np.std(data_train0, axis=0)
data_train0 = (data_train0 - avg) / std
data_test0 = (data_test0 - avg) / std

Wir fügen die Spalte mit konstanden Einsen hinzu:

In [ ]:
data_train = np.ones( (np.shape(data_train0)[0], np.shape(data_train0)[1]+1) )
data_test  = np.ones( (np.shape(data_test0)[0], np.shape(data_test0)[1]+1) )
data_train[:,1:] = data_train0
data_test[:,1:] = data_test0

**Logistische Regression (eigene Implemetierung)**

In [ ]:
num_features = np.shape(data_train)[1]

Ein Vektor für die Modellparameter:

In [ ]:
w = np.zeros(num_features)

Die Hypothesefunktion: basierend auf die Modellparameter $w$ berechnet diese Funktion die Schätzung der Wahrscheinlichkeit der positiven Klasse im Falle eines Instanzen $x$.

In [ ]:
def pred(w, x):
  return 1/(1+np.exp(-np.dot(w,x)))

Trainingszyklus: Optimierung der Modellparameter mit stochastischem Gradientenabstieg.

In [ ]:
num_epochs = 1000
lr = 1e-3

all_w = []
# Wir speichern die Modellparameter, damit wir später anzeigen können,
# wie sich die Modellparameter während des Trainings ändern

for i in range(num_epochs):
    for j in range(len(data_train)):
      y_pred = pred(w, data_train[j])
      w = w - lr*(y_pred-labels_train[j])*data_train[j]

    all_w.append(w)

Die Schätzungen (Prediktionen) für die Testdaten:

In [ ]:
predictions = []
for j in range(len(data_test)):
  y_pred = np.round(pred(w, data_test[j]))
  predictions.append(y_pred)

Wir berechnen, wie genau die Prediktionen sind. Der Anteil der korrekten Prediktionen ist:

In [ ]:
np.mean(predictions==labels_test)

Wir sehen an, wie sich die Gewichte (Modellparameter) während des Trainings ändern:

In [ ]:
all_w = np.array(all_w)

plt.plot( all_w[:,0], label = "w0" )
plt.plot( all_w[:,1], label = "w1" )
plt.plot( all_w[:,2], label = "w2" )

plt.show()

In [ ]:
plt.plot( all_w[:100,5], label = "w5" )
plt.plot( all_w[:100,7], label = "w7" )
plt.plot( all_w[:100,10], label = "w10" )

plt.show()

Darstellung der Gewichte der am Ende des Trainings:

In [ ]:
sns.set_theme(style="whitegrid")

f, ax = plt.subplots(figsize=(5, 20))

column_weights = pd.DataFrame.from_dict({'column': column_names, 'w': w[1:] })
sns.barplot(x="w", y="column", data=column_weights, color="b", orient='h')
ax.set(ylabel="Column", xlabel="weight")
ax.grid(False)

**Logistische Regression in der Bibliothek "scikit-learn"**

In [ ]:
model = LogisticRegression()
model.fit(data_train, labels_train)

pred = model.predict(data_test)

np.sum(pred == labels_test)/len(labels_test)

**Mehrere Klassen**

(Erkennung handgeschriebenen Ziffern)

In [ ]:
data = np.loadtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/semeion/semeion.data')

In [ ]:
image_size=(16,16)

tb = widgets.TabBar([str(i) for i in range(10)], location='start')
for i in range(10):
  with tb.output_to(i):
    an_image = np.reshape(data[i*20,0:256], image_size )
    plt.imshow(an_image)
    plt.show()

In [ ]:
# Vorbereitung der Daten

data_train = data[:1093,0:256]
labels_train = data[:1093,256:266]
data_test = data[1093:,0:256]
labels_test = data[1093:,256:266]

def ordinary_labels(raw_labels):
  o_lab = []
  for i in range(len(raw_labels)):
    o_lab.append( np.argmax(raw_labels[i,:]) )
  return np.array(o_lab)

labels_train = ordinary_labels(labels_train)
labels_test  = ordinary_labels(labels_test)

In [ ]:
# Wenn wir die logistische Regression aus scikit-learn benutzen,
# brauchen wir uns keine Sorgen wegen der Anzahl der Klassen zu machen

model = LogisticRegression()
model.fit(data_train, labels_train)

pred = model.predict(data_test)

np.sum(pred == labels_test)/len(labels_test)